# Z Estimator

## Importing libraries

In [ ]:
# Libraries to help with reading and manipulating data
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

## Extracting the dataset

In [ ]:
df_train_prop2016 = pd.read_csv('properties_2016.csv')
df_train_prop2016.head()

In [ ]:
df_train_prop2016.shape
print(r'The dataset has {rownum} rows and {colnum} columns'.format(rownum = df_train_prop2016.shape[0], colnum = df_train_prop2016.shape[1]))

- There's no whole row with a missing value

## Exploratory Data Analysis

In [8]:
# profile = ProfileReport(
#     df_train_prop2016, 
#     title = "Profiling Report - Train Dataset",
#     dataset={
#         "description": "Profiling Report for train dataset",
#         "author": "Nestor Arias",
#         "copyright_holder": "AI Caramba !!",
#         "copyright_year": 2024
#     }
# )
# profile.to_notebook_iframe()